<a href="https://colab.research.google.com/github/jangjung-coding/lettuce_predict_ai/blob/main/lettuce_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 초기 설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [71]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

fc-cache: succeeded


In [81]:
import random
import pandas as pd
import numpy as np
import os
import glob

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore')

In [82]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [90]:
CFG = {
    'EPOCHS':30,
    'LEARNING_RATE':1e-3,
    'BATCH_SIZE':16,
    'SEED':41
}

In [91]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [97]:
%cd /content/drive/MyDrive/googleCloud AI 인재양성 교육/[프로젝트]상추의 생육 환경 예측 AI/data

/content/drive/MyDrive/googleCloud AI 인재양성 교육/[프로젝트]상추의 생육 환경 예측 AI/data


In [99]:
all_input_list = sorted(glob.glob('./train_input/*.csv'))
all_target_list = sorted(glob.glob('./train_target/*.csv'))

In [101]:
train_input_list = all_input_list[:25]
train_target_list = all_target_list[:25]

val_input_list = all_input_list[25:]
val_target_list = all_target_list[25:]

In [102]:
class CustomDataset(Dataset):
    def __init__(self, input_paths, target_paths, infer_mode):
        self.input_paths = input_paths
        self.target_paths = target_paths
        self.infer_mode = infer_mode

        self.data_list = []
        self.label_list = []
        print('Data Pre-processing..')
        for input_path, target_path in tqdm(zip(self.input_paths, self.target_paths)):
            input_df = pd.read_csv(input_path)
            target_df = pd.read_csv(target_path)

            input_df = input_df.drop(columns=['obs_time'])
            input_df = input_df.fillna(0)

            input_length = int(len(input_df)/24)
            target_length = int(len(target_df))

            for idx in range(target_length):
                time_series = input_df[24*idx:24*(idx+1)].values
                self.data_list.append(torch.Tensor(time_series))
            for label in target_df["predicted_weight_g"]:
                self.label_list.append(label)
        print('Done.')

    def __getitem__(self, index):
        data = self.data_list[index]
        label = self.label_list[index]
        if self.infer_mode == False:
            return data, label
        else:
            return data

    def __len__(self):
        return len(self.data_list)

In [103]:
train_dataset = CustomDataset(train_input_list, train_target_list, False)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=6)

val_dataset = CustomDataset(val_input_list, val_target_list, False)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=6)

Data Pre-processing..


0it [00:00, ?it/s]

Done.
Data Pre-processing..


0it [00:00, ?it/s]

Done.


In [104]:
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.lstm = nn.LSTM(input_size=15, hidden_size=256, batch_first=True, bidirectional=False)
        self.classifier = nn.Sequential(
            nn.Linear(256, 1),
        )

    def forward(self, x):
        hidden, _ = self.lstm(x)
        output = self.classifier(hidden[:,-1,:])
        return output

In [105]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.L1Loss().to(device)

    best_loss = 9999
    best_model = None
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for X, Y in iter(train_loader):
            X = X.to(device)
            Y = Y.to(device)

            optimizer.zero_grad()

            output = model(X)
            loss = criterion(output, Y)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        val_loss = validation(model, val_loader, criterion, device)

        print(f'Train Loss : [{np.mean(train_loss):.5f}] Valid Loss : [{val_loss:.5f}]')

        if scheduler is not None:
            scheduler.step(val_loss)

        if best_loss > val_loss:
            best_loss = val_loss
            best_model = model
    return best_model

In [106]:
def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []
    with torch.no_grad():
        for X, Y in iter(val_loader):
            X = X.float().to(device)
            Y = Y.float().to(device)

            model_pred = model(X)
            loss = criterion(model_pred, Y)

            val_loss.append(loss.item())

    return np.mean(val_loss)

In [107]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, threshold_mode='abs',min_lr=1e-8, verbose=True)

best_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

Train Loss : [34.78162] Valid Loss : [27.60703]
Train Loss : [32.56001] Valid Loss : [26.14123]
Train Loss : [31.45942] Valid Loss : [25.45856]
Train Loss : [31.01905] Valid Loss : [25.13085]
Train Loss : [30.51982] Valid Loss : [24.90011]
Train Loss : [30.47947] Valid Loss : [24.86339]
Train Loss : [30.31722] Valid Loss : [24.82071]
Train Loss : [30.33345] Valid Loss : [24.81744]
Train Loss : [30.34926] Valid Loss : [24.80219]
Train Loss : [30.36699] Valid Loss : [24.79919]
Train Loss : [30.17920] Valid Loss : [24.80735]
Train Loss : [30.21411] Valid Loss : [24.78642]
Train Loss : [30.25776] Valid Loss : [24.79602]
Train Loss : [30.29879] Valid Loss : [24.78557]
Train Loss : [30.28586] Valid Loss : [24.78725]
Train Loss : [30.24407] Valid Loss : [24.79298]
Train Loss : [30.31077] Valid Loss : [24.80108]
Train Loss : [30.23780] Valid Loss : [24.80370]
Train Loss : [30.25825] Valid Loss : [24.82526]
Train Loss : [30.25785] Valid Loss : [24.79629]
Train Loss : [30.24134] Valid Loss : [24

In [ ]:
test_input_list = sorted(glob.glob('./test_input/*.csv'))
test_target_list = sorted(glob.glob('./test_target/*.csv'))

In [ ]:
def inference_per_case(model, test_loader, test_path, device):
    model.to(device)
    model.eval()
    pred_list = []
    with torch.no_grad():
        for X in iter(test_loader):
            X = X.float().to(device)

            model_pred = model(X)

            model_pred = model_pred.cpu().numpy().reshape(-1).tolist()

            pred_list += model_pred

    submit_df = pd.read_csv(test_path)
    submit_df['predicted_weight_g'] = pred_list
    submit_df.to_csv(test_path, index=False)

In [ ]:
for test_input_path, test_target_path in zip(test_input_list, test_target_list):
    test_dataset = CustomDataset([test_input_path], [test_target_path], True)
    test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
    inference_per_case(best_model, test_loader, test_target_path, device)

In [ ]:
import zipfile
os.chdir("./test_target/")
submission = zipfile.ZipFile("../submission.zip", 'w')
for path in test_target_list:
    path = path.split('/')[-1]
    submission.write(path)
submission.close()

# Modeling
- LSTM : 순환 신경망(RNN)의 일종으로, 시간의 흐름에 따라 변하는 시계열 데이터를 잘 다룰 수 있는 모델
- XGBoost : 결정 트리 기반의 앙상블 학습 모델
- LightGBM : XGBoost와 유사한 앙상블 학습 기법이지만, 더 빠르고 효율적인 학습을 위해 최적화된 모델
- SegRNN : 시계열 데이터의 특정 구간을 분할하여 각 구간의 특징을 학습하는 모델
- Random Forest Regression : 여러 개의 결정 트리를 사용하여 예측을 수행하는 앙상블 학습 모델

참고

https://paperswithcode.com/area/time-series (모델 설명)

https://towardsdatascience.com/ensemble-models-5a62d4f4cb0c (앙상블 모델 설명)

## 1. LSTM (Long Short-Term Memory)
LSTM은 장기 의존성 문제를 해결하는 순환 신경망

장점:
- 시계열 데이터의 장기 패턴을 잘 학습함
- 과거 데이터의 중요한 정보를 오래 유지

단점:
- 학습 시간이 길어질 수 있음
- 많은 하이퍼파라미터로 인해 튜닝이 복잡할 수 있음

## 2. XGBoost (Extreme Gradient Boosting)
XGBoost는 결정 트리 기반의 강력한 앙상블 학습 기법입니다.

장점:
- 빠른 연산 속도와 높은 예측 성능을 제공
- 과적합 방지를 위한 다양한 규제 기법을 포함

단점:
- 큰 데이터셋에서는 메모리 사용량이 많다
- 하이퍼파라미터 튜닝이 복잡할 수 있다

## 3. LightGBM (Light Gradient Boosting Machine)
LightGBM은 대용량 데이터셋에서 빠르고 효율적인 학습을 위한 앙상블 기법입니다.

장점:
- 학습 속도가 매우 빠름
- 메모리 효율성이 높음

단점:

- 작은 데이터셋에서는 성능이 저하될 수 있음
- 데이터 전처리와 형식에 민감할 수 있음

##4. SegRNN (Segmental Recurrent Neural Network)

SegRNN은 시계열 데이터를 특정 구간으로 나누어 학습하는 순환 신경망

장점:

- 구간별 특징을 잘 반영하여 예측 정확도가 높다
- 특정 이벤트나 패턴을 더 잘 캡처할 수 있다

단점:

- 데이터 구간 설정이 어려울 수 있다
- 계산 복잡도가 높아질 수 있다

## 5. Random Forest Regression
장점:

- 여러 트리의 예측을 평균화하기 때문에 과적합 가능성이 낮다
- 구현이 비교적 간단하며, 기본 설정으로도 좋은 성능을 보임

단점:
- 많은 트리를 생성하므로 메모리 사용량이 많을 수 있음
- 많은 트리를 거쳐 예측하기 때문에 실시간 예측에는 부적합할 수 있음

# BenchMark